In [1]:
%cd /workspace/Walk_Video_PyTorch/project

from main import get_parameters
from IPython.display import clear_output

config, unkonwn = get_parameters()

config.clip_duration = 2
config.num_workers = 3
config.batch_size = 4
config.gpu_num = 1
config.img_size = 256

clear_output()

In [2]:
from typing import Any, Callable, Dict, Optional, Type

import os 
import torch 
import pytorchvideo
from pytorchvideo.data.clip_sampling import ClipSampler

from pytorchvideo.data.labeled_video_dataset import LabeledVideoDataset
from pytorchvideo.data import labeled_video_dataset, make_clip_sampler

In [3]:
def WalkDataset(
    data_path: str,
    clip_sampler: ClipSampler,
    video_sampler: Type[torch.utils.data.Sampler] = torch.utils.data.RandomSampler,
    transform: Optional[Callable[[Dict[str, Any]], Dict[str, Any]]] = None,
    video_path_prefix: str = "",
    decode_audio: bool = False,
    decoder: str = "pyav",
) -> LabeledVideoDataset:
    '''
    A helper function to create "LabeledVideoDataset" object for the Walk dataset.

    Args:
        data_path (str): Path to the data. The path defines how the data should be read. For a directory, the directory structure defines the classes (i.e. each subdirectory is class).
        clip_sampler (ClipSampler): Defines how clips should be sampled from each video. See the clip sampling documentation for more information.
        video_sampler (Type[torch.utils.data.Sampler], optional): Sampler for the internal video container. Defaults to torch.utils.data.RandomSampler.
        transform (Optional[Callable[[Dict[str, Any]], Dict[str, Any]]], optional): This callable is evaluated on the clip output before the clip is returned. Defaults to None.
        video_path_prefix (str, optional): Path to root directory with the videos that are
                loaded in ``LabeledVideoDataset``. Defaults to "".
        decode_audio (bool, optional): If True, also decode audio from video. Defaults to False. Notice that, if Ture will trigger the stack error.
        decoder (str, optional): Defines what type of decoder used to decode a video. Defaults to "pyav".

    Returns:
        LabeledVideoDataset: _description_
    '''
    return labeled_video_dataset(
        data_path,
        clip_sampler,
        video_sampler,
        transform,
        video_path_prefix,
        decode_audio,
        decoder
    )


In [4]:
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize,
    RemoveKey,
    ShortSideScale,
    RandomShortSideScale,
    UniformTemporalSubsample,
    Div255,
    RandomResizedCrop,
    create_video_transform
)

from torchvision.transforms import (
    Compose,
    Lambda,
    RandomCrop,
    RandomHorizontalFlip,
    Resize,
    ToTensor,
    
)

In [7]:
from pyexpat import model
from venv import create


class WalkDataModule():
    def __init__(self, opt):
        super().__init__()
        self._DATA_PATH = opt.split_data_path
        self._CLIP_DURATION = opt.clip_duration
        self._BATCH_SIZE = opt.batch_size
        self._NUM_WORKERS = opt.num_workers
    
    def train_dataloader(self):
        '''
        create the Walk train partition from the list of video labels 
        in directory and subdirectory. Add transform that subsamples and 
        normalizes the video before applying the scale, crop and flip augmentations.
        '''        
        train_transform = Compose(
            [
                ApplyTransformToKey(
                    key="video",
                    transform=Compose(
                        [
                            # 
                            UniformTemporalSubsample(30),
                            Div255(),
                            Normalize((0.45, 0.45, 0.45), (0.225, 0.225, 0.225)),
                            
                            # RandomShortSideScale(min_size=256, max_size=320),
                            # ShortSideScale(256),
                            # RandomCrop(244),
                            Resize(size=[256, 256]),
                            # RandomResizedCrop(512, 512, scale=0.5, aspect_ratio=0.5),
                            RandomHorizontalFlip(p=0.5),
                        ]
                    ),
                ),
                # create_video_transform(mode='train', video_key='video', num_samples=8, crop_size=(256,256), convert_to_float=False)
            ]
        )

        # train_transform = create_video_transform(mode='train', num_samples=10, crop_size=(256, 256))

        train_dataset = WalkDataset(
            data_path= os.path.join(self._DATA_PATH, 'train'),
            clip_sampler=make_clip_sampler("uniform", self._CLIP_DURATION),
            transform=train_transform
        )

        return torch.utils.data.DataLoader(
            train_dataset,
            batch_size = self._BATCH_SIZE,
            num_workers = self._NUM_WORKERS,
        )

In [8]:
data_module = WalkDataModule(config).train_dataloader()

batch = next(iter(data_module))

In [ ]:
batch['video_name'][0]

batch_value = {}

batch.keys()

for keys in batch.keys():
    if keys == 'video':
        batch_value[keys] = batch[keys][0].size()
    else:
        batch_value[keys] = batch[keys][0]

batch_value

In [ ]:
batch['video_name']

In [ ]:
batch["video"][0].permute(1, 2, 3, 0)[0].shape

batch['video'][0][0][0][0]


In [ ]:
import matplotlib.pylab as plt 

print(batch['video_name'][0])

plt.figure(figsize=(150, 150))

for i in range(batch['video'][0].size()[1]):
    plt.title(batch['video_name'][0])
    plt.subplot(1, batch['video'][0].size()[1], i + 1)
    plt.imshow(batch['video'][0].permute(1, 2, 3, 0)[i])

    plt.axis("off")

plt.show()




In [ ]:
# import matplotlib.pylab as plt 

# print(batch['video_name'][0])

# location = 1

# plt.figure(figsize=(244, 244))

# for num in range(batch['video'].size()[0]): # batch size
#     for i in range(batch['video'].size()[2]): # 帧数
#         plt.title(batch['video_name'][num])
            
#         plt.imshow(batch["video"][num].permute(1, 2, 3, 0)[i])

#         location +=1 
#         plt.axis("off")

# plt.show()

